In [1]:
# Install keras, tensorflow, and python same as the original git

!pip install keras==2.1.5
!pip install tensorflow==1.6.0

     |████████████████████████████████| 337kB 2.8MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
     |████████████████████████████████| 45.9MB 91kB/s 
     |████████████████████████████████| 3.1MB 25.3MB/s 
     |████████████████████████████████| 890kB 47.2MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=cc038a6bc77d8550ee7a7219a1455d48b95f1aedbf93df80bc6f45525df3e5f6
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorb

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
cd 'drive/My Drive/Object_Detection_02'

/content/drive/My Drive/Object_Detection_02


In [81]:
from numpy import loadtxt, expand_dims
from keras import backend as K
from keras.layers import Input
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
from imageio import imread

from yolo3.model import yolo_head, box_iou
import tensorflow as tf
import numpy as np
import cv2

In [66]:
# frame 추출 및 저장
vidcap = cv2.VideoCapture('Sample_result_video/20200706_2.mp4') # 왜 저장 안되는가? 갑자기됐넹 뭐지
success, imagefile = vidcap.read()
count = 0

while success:
    if(count%6==0):
        cv2.imwrite("Frames/20200706_3/frame%d.jpg" % count, imagefile)
        print("success")
    success, imagefile = vidcap.read()
    count+=1
#============================

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [83]:
anchors_path = 'model_data/tiny_yolo_anchors.txt'
classes_path = 'classes.txt'
weights_path = 'tiny_yolo_crack_20200623.h5'
adam = Adam(lr=1e-4)

# load the model
model = load_model('tiny_yolo_crack_20200623.h5', {'yolo_head':yolo_head, 'tf':tf, 'box_iou':box_iou}, False)

input_height = 416
input_width = 416

orig_images = [] # 원본 이미지
input_images = [] # 변환돼서 넣어질 input 이미지

frames_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frames_count)

def load_images(filename, shape):
    img = load_img(filename)
    height, width = img.size
    orig_images.append(imread(filename))
    img = load_img(filename, target_size = shape)
    img = img_to_array(img) # convert to numpy array
    img = img.astype('float32') # scale pixel values to [0, 1]
    img /= 255.0
    img = expand_dims(img, 0)
    #img = np.array(img)
    return img, height, width
        
for i in range(0, frames_count):
    if(i%6==0): #18로 바꾸면 뭔가 되긴 하는데, ValueError: Error when checking : expected input_2 to have 5 dimensions, but got array with shape (1, 416, 416, 3) 에러가 뜸. >> nput_images를 np.array()하면 5D가 되긴 함.
        img_filename = 'Frames/20200706_3/frame%d.jpg'%i
        img, img_height, img_width = load_images(img_filename, (input_height, input_width))
        input_images.append(img)

orig_images = np.array(orig_images)
input_images = np.array(input_images)
print(input_images)
yhat = model.predict([orig_images, input_images, input_images])
print([a.shape for a in yhat])

# 동영상에서 frame 추출한 다음에 저거 article 과정 따라하면 될거같은데?? for문으로 frame마다 이미지로 지정해서 반복하고....

378
[[[[[0.5529412  0.53333336 0.41568628]
    [0.6039216  0.4392157  0.4862745 ]
    [0.5921569  0.49019608 0.5019608 ]
    ...
    [0.59607846 0.44313726 0.32156864]
    [0.59607846 0.44313726 0.32156864]
    [0.59607846 0.44313726 0.32156864]]

   [[0.5921569  0.54509807 0.4509804 ]
    [0.654902   0.45490196 0.49019608]
    [0.61960787 0.4745098  0.4392157 ]
    ...
    [0.59607846 0.44313726 0.32156864]
    [0.59607846 0.44313726 0.32156864]
    [0.59607846 0.44313726 0.32156864]]

   [[0.5686275  0.49019608 0.39215687]
    [0.69411767 0.4627451  0.47058824]
    [0.6745098  0.49411765 0.43529412]
    ...
    [0.59607846 0.44313726 0.32156864]
    [0.59607846 0.44313726 0.32156864]
    [0.59607846 0.44313726 0.32156864]]

   ...

   [[0.5803922  0.49019608 0.41960785]
    [0.5882353  0.49803922 0.42745098]
    [0.5921569  0.5019608  0.43137255]
    ...
    [0.74509805 0.6666667  0.5294118 ]
    [0.7764706  0.69803923 0.56078434]
    [0.8039216  0.7254902  0.5803922 ]]

   [[0.6    

ValueError: ignored